# TASK 5

Создайте соединение с базой данных с помощью библиотеки sqlite3.

In [1]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
conn = sq.connect('/content/drive/MyDrive/Google_Drive/School21/Day11/checking-logs.sqlite')

Используя только один запрос для каждой из групп, создайте два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.
times должно иметь значения before и after.
avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
Учитываются только те пользователи, для которых имеются наблюдения и before, и after.
Мы по-прежнему не используем лабораторное задание 'project1'.

In [43]:
pd.io.sql.read_sql('PRAGMA table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [44]:
pd.io.sql.read_sql('PRAGMA table_info(control)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [55]:
test_result = pd.read_sql("""SELECT time, avg(diff)
                        FROM 
                          (SELECT uid, time, avg(diff) as diff
                          FROM 
                            (SELECT uid, (strftime('%s', first_commit_ts) - deadlines)/60/60 diff,
                            case first_commit_ts > first_view_ts
                              when 1 then 'after'
                              else 'before'
                            end time
                          FROM test
                          JOIN deadlines on labname = labs
                          WHERE
                            labs not like 'project1'
                            and uid in
                            (SELECT uid 
                            FROM test
                            WHERE first_commit_ts > first_view_ts and labname not like 'project1'
                        intersect
                        SELECT uid
                        FROM test
                        WHERE first_commit_ts <= first_view_ts and labname not like 'project1'))
                        GROUP BY uid, time)
                        GROUP BY time
                        """, conn)
test_result

,time,avg(diff)
0,after,-99.523810
1,before,-66.047619


In [56]:
test_control = pd.read_sql("""SELECT time, avg(diff)
                        FROM 
                          (SELECT uid, time, avg(diff) as diff
                          FROM 
                            (SELECT uid, (strftime('%s', first_commit_ts) - deadlines)/60/60 diff,
                            case first_commit_ts > first_view_ts
                              when 1 then 'after'
                              else 'before'
                            end time
                          FROM control
                          JOIN deadlines on labname = labs
                          WHERE
                            labs not like 'project1'
                            and uid in
                            (SELECT uid 
                            FROM control
                            WHERE first_commit_ts > first_view_ts and labname not like 'project1'
                        intersect
                        SELECT uid
                        FROM control
                        WHERE first_commit_ts <= first_view_ts and labname not like 'project1'))
                        GROUP BY uid, time)
                        GROUP BY time
                        """, conn)
test_control

,time,avg(diff)
0,after,-99.322222
1,before,-98.033333


Закройте соединение с базой данных.

In [41]:
conn.close()

Наличие новостной ленты влияет на поведение участников